In [17]:
#%pip install --upgrade git+https://github.com/OpenConjecture/pyorient.git

  Cloning https://github.com/OpenConjecture/pyorient.git to /private/var/folders/5c/4n982b9s0svctvnqygs9sbxh0000gn/T/pip-req-build-9vbovzwt
  Running command git clone --filter=blob:none --quiet https://github.com/OpenConjecture/pyorient.git /private/var/folders/5c/4n982b9s0svctvnqygs9sbxh0000gn/T/pip-req-build-9vbovzwt
  Resolved https://github.com/OpenConjecture/pyorient.git to commit c41c3b615fad09ea9dc71b802339c865d5cdce14
  Preparing metadata (setup.py) ... done
  Created wheel for pyorient: filename=pyorient-1.5.5-py3-none-any.whl size=83239 sha256=3c4501c718b74297d17d31b0213352956515cee15a73ffc1f2a78d4cbc34a4c5
  Stored in directory: /private/var/folders/5c/4n982b9s0svctvnqygs9sbxh0000gn/T/pip-ephem-wheel-cache-2z4t75rr/wheels/3f/dd/1c/7382ed6d7620fea508cd831fada5b14f804ed0fb5cdaa89b34
Successfully built pyorient
Note: you may need to restart the kernel to use updated packages.


In [8]:
import socket
import pandas as pd
import pyorient #La libreria está muy vieja por lo cual puede tener errores, no se actualiza desde hace 6 años 

In [4]:
#Nos conectamos al servidor/localhost

client = pyorient.OrientDB("localhost", 2424)
client.set_session_token( True )  # set true to enable the token based
client.db_open( "Aero", "root", "root" )

In [9]:
#Importamos las bases de datos, igualmente a como lo hicimos en ArangoDB

aeropuerto_info = pd.read_csv("DataFrames/Info_Basica_Aeropuertos.csv").iloc[:,1:]
arrivos = pd.read_csv("DataFrames/arrivos.csv").iloc[:,1:]
aerolineas = pd.read_csv("DataFrames/Aerolineas.csv").iloc[:,1:]
aerolineas.fillna("NO",inplace=True)
aerolineas.drop_duplicates(subset=['IATA'],inplace=True)
aerolineas = aerolineas[aerolineas['IATA'].str.len() <= 2]
arrivos = arrivos.applymap(str)
arrivos.drop_duplicates(inplace=True,subset=['Numero de Vuelo'])
arrivos["cod origen"] = arrivos.apply(lambda x: x['Origen'].split()[-1].replace('[','').replace(']',''), axis=1)
arrivos['cod destino'] = arrivos.apply(lambda x: x['Destino'].split()[-1].replace('(','').replace(')',''), axis=1)
aeropuerto_info['cod aero'] = aeropuerto_info.apply(lambda x: x['Nombre'].split()[-1].replace('(','').replace(')',''), axis=1)
arrivos['cod vuelo'] = arrivos['Numero de Vuelo'].apply(lambda a: a[0:2]+a[3:])
arrivos['IATA'] = arrivos['Numero de Vuelo'].apply(lambda a: a[0:2])

In [12]:
#Ahora proseguimos a crear las clases

client.command("create class aeropuerto extends V") #Tira error pero sin embargo se creo en el localhost
client.command("create class aerolinea extends V") #Tira error pero sin embargo se creo en el localhost
client.command("create class arrivos extends V") #Tira error pero sin embargo se creo en el localhost

KeyboardInterrupt: 

In [22]:
#Ahora teniendo ya las clases proseguimos a utilizar el insert para agregar la información 

##Primero a la clase de aeropuerto
for row in range(aeropuerto_info.shape[0]):
    
    client.command(f"insert into aeropuerto set {aeropuerto_info.columns[0]} = {aeropuerto_info.iloc[row,0]}, {aeropuerto_info.columns[1]} = {aeropuerto_info.iloc[row,1]}, {aeropuerto_info.columns[2]} = {aeropuerto_info.iloc[row,2]}, {aeropuerto_info.columns[3]} = {aeropuerto_info.iloc[row,3]}")

PyOrientCommandException:  - 

In [14]:
# Como se puede observar en el error que hay probablemente sea que OrientDB no soporta carácteres en español

In [16]:
aeropuerto_info.rename(columns={"Ubicación":"Ubicacion","Teléfono":"Telefono"},inplace=True)

In [28]:
for row in range(aeropuerto_info.shape[0]):
    
    client.command(f"insert into aeropuerto set {aeropuerto_info.columns[0]} = {aeropuerto_info.iloc[row,0]}, {aeropuerto_info.columns[1]} = {aeropuerto_info.iloc[row,1]}, {aeropuerto_info.columns[2]} = {aeropuerto_info.iloc[row,2]}, {aeropuerto_info.columns[3]} = {aeropuerto_info.iloc[row,3]}")

TimeoutError: timed out

In [35]:
aeropuerto_info

,Nombre,Ubicacion,Timezone,Telefono,cod aero
0,Chicago O'hare Airport (ORD),"10000 W O'Hare Ave, Chicago, Illinois 60666, ...",(UTC/GMT -6),+1 800-832-6352,ORD
1,San Salvador Cuscatlan Airport (SAL),"RN-5, San Salvador, El Salvador",(UTC/GMT -6),+503 2349 9455,SAL
2,Panama City Albrook Marcos A. Gelabert Airport...,"Avenida Canfield, Panama, Panama",(UTC/GMT -5),+507 501-9272,PAC
3,Guatemala City La Aurora Airport (GUA),"Guatemala City, Guatemala",(UTC/GMT -6),+502 2321 5000,GUA
4,Puerto Rico Airport (PUR),"Avenida Aeropuerto, 979 Carolina, Puerto Rico",(UTC/GMT -4),+1 787-253-2329,PUR
5,Belize City Municipal Airport (TZA),"Belize City, Belize",(UTC/GMT -6),+501 224-5671,TZA
6,Cap Haitien Cap Haitien Airport (CAP),"Cap-Haitien, Haiti",(UTC/GMT -5),+509 44 80 5107,CAP
7,Montevideo Carrasco Airport (MVD),"101, 14000 Ciudad de la Costa, Uruguay",(UTC/GMT -2),+598 2604 0329,MVD
8,San Juan Luis Munoz Marin Airport (SJU),"Avenida Aeropuerto, 979 Carolina, Puerto Rico",(UTC/GMT -4),+1 787-253-2329,SJU
9,Santo Domingo Las Americas Airport (SDQ),"Ruta 66 , Santo Domingo, Dominican Republic",(UTC/GMT -4),+1 809-947-2225,SDQ


In [30]:
aeropuerto_info['Nombre'] = aeropuerto_info['Nombre'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [33]:
client.command(f"insert into aeropuerto set {aeropuerto_info.columns[0]} = {aeropuerto_info.iloc[0,0]}, {aeropuerto_info.columns[1]} = {aeropuerto_info.iloc[0,1]}, {aeropuerto_info.columns[2]} = {aeropuerto_info.iloc[0,2]}, {aeropuerto_info.columns[3]} = {aeropuerto_info.iloc[0,3]}")

BlockingIOError: [Errno 35] Resource temporarily unavailable

In [34]:
client.command("insert into aeropuerto set name = Chicago O'hare Airport (ORD)")

PyOrientCommandException:  - 